<a href="https://colab.research.google.com/github/antoniod621/CRUD-App-Students/blob/main/3_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#install mrjob
!pip install mrjob

In [2]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving nashville-tweets-2019-01-28 to nashville-tweets-2019-01-28
User uploaded file "nashville-tweets-2019-01-28" with length 25039251 bytes


In [ ]:
# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [25]:
%%file 3_days.py
#write the top pragma string to save this file in local colab drive as 3_days.py
from mrjob.job import MRJob
import json

'''
Count the tweets per day.

See http://mike.teczno.com/notes/streaming-data-from-twitter.html for parsing info.
Get the screen name by accessing tweet['user']['screen_name']

Look at tweet['created_at'] for datetime of creation. Just use the first word in the date to get the day.


'''


class DaysTweets(MRJob):
    def mapper(self, _, line):
        try:
            # do something  
            tweet = json.loads(line)
            yield tweet['created_at'].split(" ")[0], 1
        except:
            pass

    def reducer(self, key, counts):
        # do something     
        yield key, sum(counts)

if __name__ == '__main__':
    DaysTweets.run()

Overwriting 3_days.py


## Step 1. Run MR program locally(Colab)

In [26]:
#write the command to run it first to check for error
%%timeit
!python 3_days.py -r local nashville-tweets-2019-01-28 > 3_days.out

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/3_days.root.20220331.165354.895062
Running step 1 of 1...
job output is in /tmp/3_days.root.20220331.165354.895062/output
Streaming final output from /tmp/3_days.root.20220331.165354.895062/output...
Removing temp directory /tmp/3_days.root.20220331.165354.895062...
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/3_days.root.20220331.165356.948218
Running step 1 of 1...
job output is in /tmp/3_days.root.20220331.165356.948218/output
Streaming final output from /tmp/3_days.root.20220331.165356.948218/output...
Removing temp directory /tmp/3_days.root.20220331.165356.948218...
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/3_days.root.20220331.165358.888090
Running step 1 of 1...
job output is in /tmp/3_days.root.20220331.16535

In [27]:
!cat 3_days.out

"Sun"	6294


## Step 2. Run MR program on AWS EMR

### Please copy nashville-tweets-2019-01-28 into your S3 bucket. Note that if you already uploaded the data in previous step you can just use the same url here.

In [ ]:
%%timeit
!python 3_days.py -r emr s3://<s3 url of the Tweet dataset> --cloud-tmp-dir=s3://<s3 url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 3_days.out

In [ ]:
!cat 3_days.out

In [28]:
f = open('3_days.out')
lines = f.readlines()
f.close()
for line in lines:
  if line[:-1] in ['"Sun"\t6294']:
    print("passed")
  else:
    assert False

passed
